In [2]:
import pandas as pd
import numpy as np

In [72]:
class ev_trainer():
    def __init__(self, name, item, pkrus=True):
        '''Initial class ev_trainer:
        name: pokemon's name
        item: pokemon's ev traning item, take effect if is in ['hp', 'atk', 'def', 'spa', 'spd', 'spe']
        pkrus: Whether pokemon have pkrus activate or not, default is True
        ev_spread: pandas series to store ev values'''
        
        self.name = name
        self.item = item
        self.pkrus = pkrus
        self.ev_spread = pd.Series(index=['hp', 'atk', 'def', 'spa', 'spd', 'spe'], data=[0]*6)
        self.trained = []
        self.full_ev = False
        
    def status(self):
        '''Function to print pokemon status include name, item, pkrus, current ev'''
        
        print('********************************************')
        print('name:', self.name)
        print('item:', self.item)
        print('pkrus:', self.pkrus)
        print('current ev: (Trained {} times)'.format(len(self.trained)))
        print(self.ev_spread)
        print('Total:', np.sum(self.ev_spread))
        print('********************************************')
    
    def train(self, op, base=2, pr=True):
        '''Function used to train pokemon, input:
        op: opponent, currently only support ev yield of the opponent
            have to be within ['hp', 'atk', 'def', 'spa', 'spd', 'spe']
        base: base point yield by defeating that pokemon, default = 2
        pr: print status or not, default=True'''
        
        # Check if ev is full or not
        if not self.full_ev:
            if self.pkrus == True:
                pkr = 2
            else:
                pkr = 1
        
            self.update_ev(op, base*pkr)
            
            if not self.full_ev:
                if self.item in ['hp', 'atk', 'def', 'spa', 'spd', 'spe']:
                    self.update_ev(self.item, 8*pkr)
        
        self.trained.append(op)
        
        if pr:
            print('{} finished one {} training with base {} ev'.format(self.name, op, base))
            print('current ev: (Trained {} times)'.format(len(self.trained)))
            print(self.ev_spread)
            print('\n')
    
    
    def update_ev(self, ev, value):
        '''Function to update value amount of ev to ev stats, ev have to be in ['hp', 'atk', 'def', 'spa', 'spd', 'spe']'''
        
        total = np.sum(self.ev_spread)
        pr_tot = total + value
        pr_ev = self.ev_spread[ev] + value
        if pr_tot <= 510 and pr_ev <= 252:
            self.ev_spread[ev] += value
        else:
            self.ev_spread[ev] += np.min([510 - total, 252 - self.ev_spread[ev]])
        
        if np.sum(self.ev_spread) >= 510:
            self.full_ev = True
        
    def clear(self, how='all', amount=1, value = 10):
        '''Clear pokemon's ev, take input:
        how: how to delete, can be 'all' or specific stats in ['hp', 'atk', 'def', 'spa', 'spd', 'spe']
        amount: if how is one specfic stats, assuming you are using berries, select amount of berries your pokemon is eating
        value: value of ev reduced per berry, default is 10'''
        
        if how == 'all':
            self.ev_spread = pd.Series(index=['hp', 'atk', 'def', 'spa', 'spd', 'spe'], data=[0]*6)
        elif how in self.ev_spread.index:
            self.ev_spread[how] -= amount * value
            if self.ev_spread[how] <= 0:
                self.ev_spread[how] = 0
            
    def change_item(self, item):
        '''function used to change item'''
        self.item = item
    
    def cont_train(self, opl, base=2):
        ''' For continue training, support same base point only, default = 2
        input is a list of traning opponent for pokemon to train automatically one by one, print status after the last opponent
        from opponent list opl'''
        
        for i, op in enumerate(opl):
            pr = False
            if i == len(opl)-1:
                pr = True
            self.train(op, base, pr=pr)
            
    def start_training(self):
        '''Interactive function to help fast training versus various opponent, 
        only support base point = 2 opponent for fast finishing
        IMO best used when trying to quickly max out 2 stats within same range (e.g. Route 7 for atk and spd)'''
        
        print('You entered continuous training mode with {}'.format(self.name))
        comd = 'a'
        while comd != 'q':
            self.status()
            print('Pick your opponent (choose number with keyboard):')
            print('1. Hp   2. Atk   3. Def   4. Spa   5. Spd   6. Spe   7. Change item')
            comd = input()
            if comd == '7':
                self.item = input('Enter training item or None: ')
            cmd_dict = {'1': 'hp', '2': 'atk', '3': 'def', '4': 'spa', '5': 'spd', '6': 'spe'}
            if comd in cmd_dict.keys():
                self.train(cmd_dict[comd], pr=False)

Below is example for training Kubfu

In [73]:
Kubfu = ev_trainer('Kubfu', 'spe')

In [56]:
Kubfu.status()

********************************************
name: Kubfu
item: spe
pkrus: True
current ev: (Trained 0 times)
hp     0
atk    0
def    0
spa    0
spd    0
spe    0
dtype: int64
Total: 0
********************************************


In [57]:
Kubfu.cont_train(['atk', 'spe', 'atk', 'atk', 'atk', 'spe'])

Kubfu finished one spe training with base 2 ev
current ev: (Trained 6 times)
hp       0
atk     16
def      0
spa      0
spd      0
spe    104
dtype: int64




In [58]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 7 times)
hp       0
atk     16
def      0
spa      0
spd      0
spe    124
dtype: int64




In [59]:
Kubfu.cont_train(['spe']*2)

Kubfu finished one spe training with base 2 ev
current ev: (Trained 9 times)
hp       0
atk     16
def      0
spa      0
spd      0
spe    164
dtype: int64




In [60]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 10 times)
hp       0
atk     16
def      0
spa      0
spd      0
spe    184
dtype: int64




In [61]:
Kubfu.change_item('atk')

In [62]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 11 times)
hp       0
atk     32
def      0
spa      0
spd      0
spe    188
dtype: int64




In [63]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 12 times)
hp       0
atk     48
def      0
spa      0
spd      0
spe    192
dtype: int64




In [64]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 13 times)
hp       0
atk     64
def      0
spa      0
spd      0
spe    196
dtype: int64




In [65]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 14 times)
hp       0
atk     80
def      0
spa      0
spd      0
spe    200
dtype: int64




In [66]:
Kubfu.train('atk')

Kubfu finished one atk training with base 2 ev
current ev: (Trained 15 times)
hp       0
atk    100
def      0
spa      0
spd      0
spe    200
dtype: int64




In [67]:
Kubfu.train('atk')

Kubfu finished one atk training with base 2 ev
current ev: (Trained 16 times)
hp       0
atk    120
def      0
spa      0
spd      0
spe    200
dtype: int64




In [68]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 17 times)
hp       0
atk    136
def      0
spa      0
spd      0
spe    204
dtype: int64




In [69]:
Kubfu.train('spe')

Kubfu finished one spe training with base 2 ev
current ev: (Trained 18 times)
hp       0
atk    152
def      0
spa      0
spd      0
spe    208
dtype: int64




In [70]:
Kubfu.status()

********************************************
name: Kubfu
item: atk
pkrus: True
current ev: (Trained 18 times)
hp       0
atk    152
def      0
spa      0
spd      0
spe    208
dtype: int64
Total: 360
********************************************


In [71]:
Kubfu.start_training()

You entered continuous training mode with Kubfu
********************************************
name: Kubfu
item: atk
pkrus: True
current ev: (Trained 18 times)
hp       0
atk    152
def      0
spa      0
spd      0
spe    208
dtype: int64
Total: 360
********************************************
Pick your opponent (choose number with keyboard):
1. Hp   2. Atk   3. Def   4. Spa   5. Spd   6. Spe   7. Change item
6
********************************************
name: Kubfu
item: atk
pkrus: True
current ev: (Trained 19 times)
hp       0
atk    168
def      0
spa      0
spd      0
spe    212
dtype: int64
Total: 380
********************************************
Pick your opponent (choose number with keyboard):
1. Hp   2. Atk   3. Def   4. Spa   5. Spd   6. Spe   7. Change item
2
********************************************
name: Kubfu
item: atk
pkrus: True
current ev: (Trained 20 times)
hp       0
atk    188
def      0
spa      0
spd      0
spe    212
dtype: int64
Total: 400
**********************